# **New York City Yellow Taxi Data**

## Objective
In this case study you will be learning exploratory data analysis (EDA) with the help of a dataset on yellow taxi rides in New York City. This will enable you to understand why EDA is an important step in the process of data science and machine learning.

## **Problem Statement**
As an analyst at an upcoming taxi operation in NYC, you are tasked to use the 2023 taxi trip data to uncover insights that could help optimise taxi operations. The goal is to analyse patterns in the data that can inform strategic decisions to improve service efficiency, maximise revenue, and enhance passenger experience.

## Tasks
You need to perform the following steps for successfully completing this assignment:
1. Data Loading
2. Data Cleaning
3. Exploratory Analysis: Bivariate and Multivariate
4. Creating Visualisations to Support the Analysis
5. Deriving Insights and Stating Conclusions

---

**NOTE:** The marks given along with headings and sub-headings are cumulative marks for those particular headings/sub-headings.<br>

The actual marks for each task are specified within the tasks themselves.

For example, marks given with heading *2* or sub-heading *2.1* are the cumulative marks, for your reference only. <br>

The marks you will receive for completing tasks are given with the tasks.

Suppose the marks for two tasks are: 3 marks for 2.1.1 and 2 marks for 3.2.2, or
* 2.1.1 [3 marks]
* 3.2.2 [2 marks]

then, you will earn 3 marks for completing task 2.1.1 and 2 marks for completing task 3.2.2.


---

## Data Understanding
The yellow taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts.

The data is stored in Parquet format (*.parquet*). The dataset is from 2009 to 2024. However, for this assignment, we will only be using the data from 2023.

The data for each month is present in a different parquet file. You will get twelve files for each of the months in 2023.

The data was collected and provided to the NYC Taxi and Limousine Commission (TLC) by technology providers like vendors and taxi hailing apps. <br>

You can find the link to the TLC trip records page here: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

###  Data Description
You can find the data description here: [Data Dictionary](https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

**Trip Records**



|Field Name       |description |
|:----------------|:-----------|
| VendorID | A code indicating the TPEP provider that provided the record. <br> 1= Creative Mobile Technologies, LLC; <br> 2= VeriFone Inc. |
| tpep_pickup_datetime | The date and time when the meter was engaged.  |
| tpep_dropoff_datetime | The date and time when the meter was disengaged.   |
| Passenger_count | The number of passengers in the vehicle. <br> This is a driver-entered value. |
| Trip_distance | The elapsed trip distance in miles reported by the taximeter. |
| PULocationID | TLC Taxi Zone in which the taximeter was engaged |
| DOLocationID | TLC Taxi Zone in which the taximeter was disengaged |
|RateCodeID |The final rate code in effect at the end of the trip.<br> 1 = Standard rate <br> 2 = JFK <br> 3 = Newark <br>4 = Nassau or Westchester <br>5 = Negotiated fare <br>6 = Group ride |
|Store_and_fwd_flag |This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server.  <br>Y= store and forward trip <br>N= not a store and forward trip |
|Payment_type| A numeric code signifying how the passenger paid for the trip. <br> 1 = Credit card <br>2 = Cash <br>3 = No charge <br>4 = Dispute <br>5 = Unknown <br>6 = Voided trip |
|Fare_amount| The time-and-distance fare calculated by the meter. <br>Extra Miscellaneous extras and surcharges.  Currently, this only includes the 0.50 and 1 USD rush hour and overnight charges. |
|MTA_tax |0.50 USD MTA tax that is automatically triggered based on the metered rate in use. |
|Improvement_surcharge | 0.30 USD improvement surcharge assessed trips at the flag drop. The improvement surcharge began being levied in 2015. |
|Tip_amount |Tip amount – This field is automatically populated for credit card tips. Cash tips are not included. |
| Tolls_amount | Total amount of all tolls paid in trip.  |
| total_amount | The total amount charged to passengers. Does not include cash tips. |
|Congestion_Surcharge |Total amount collected in trip for NYS congestion surcharge. |
| Airport_fee | 1.25 USD for pick up only at LaGuardia and John F. Kennedy Airports|

Although the amounts of extra charges and taxes applied are specified in the data dictionary, you will see that some cases have different values of these charges in the actual data.

**Taxi Zones**

Each of the trip records contains a field corresponding to the location of the pickup or drop-off of the trip, populated by numbers ranging from 1-263.

These numbers correspond to taxi zones, which may be downloaded as a table or map/shapefile and matched to the trip records using a join.

This is covered in more detail in later sections.

---

## **1** Data Preparation

<font color = red>[5 marks]</font> <br>

### Import Libraries

In [3]:
# Import warnings
import warnings
warnings.filterwarnings("always")




In [4]:
# Import the libraries you will be using for analysis

import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._boots

In [5]:
# Recommended versions
# numpy version: 1.26.4
# pandas version: 2.2.2
# matplotlib version: 3.10.0
# seaborn version: 0.13.2

# Check versions
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("matplotlib version:", plt.matplotlib.__version__)
print("seaborn version:", sns.__version__)

numpy version: 2.0.2
pandas version: 2.2.2
matplotlib version: 3.10.0
seaborn version: 0.13.2


### **1.1** Load the dataset
<font color = red>[5 marks]</font> <br>

You will see twelve files, one for each month.

To read parquet files with Pandas, you have to follow a similar syntax as that for CSV files.

`df = pd.read_parquet('file.parquet')`

In [8]:
# Try loading one file
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/files'
files = os.listdir(file_path)

sampled_df = pd.DataFrame()

# df = pd.read_parquet('2023-1.parquet')
# df.info()

Mounted at /content/drive


How many rows are there? Do you think handling such a large number of rows is computationally feasible when we have to combine the data for all twelve months into one?

To handle this, we need to sample a fraction of data from each of the files. How to go about that? Think of a way to select only some portion of the data from each month's file that accurately represents the trends.

#### Sampling the Data
> One way is to take a small percentage of entries for pickup in every hour of a date. So, for all the days in a month, we can iterate through the hours and select 5% values randomly from those. Use `tpep_pickup_datetime` for this. Separate date and hour from the datetime values and then for each date, select some fraction of trips for each of the 24 hours.

To sample data, you can use the `sample()` method. Follow this syntax:

```Python
# sampled_data is an empty DF to keep appending sampled data of each hour
# hour_data is the DF of entries for an hour 'X' on a date 'Y'

sample = hour_data.sample(frac = 0.05, random_state = 42)
# sample 0.05 of the hour_data
# random_state is just a seed for sampling, you can define it yourself

sampled_data = pd.concat([sampled_data, sample]) # adding data for this hour to the DF
```

This *sampled_data* will contain 5% values selected at random from each hour.

Note that the code given above is only the part that will be used for sampling and not the complete code required for sampling and combining the data files.

Keep in mind that you sample by date AND hour, not just hour. (Why?)

---

**1.1.1** <font color = red>[5 marks]</font> <br>
Figure out how to sample and combine the files.

**Note:** It is not mandatory to use the method specified above. While sampling, you only need to make sure that your sampled data represents the overall data of all the months accurately.

In [ ]:
# Sample the data
# It is recommmended to not load all the files at once to avoid memory overload

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
# Take a small percentage of entries from each hour of every date.
# Iterating through the monthly data:
#   read a month file -> day -> hour: append sampled data -> move to next hour -> move to next day after 24 hours -> move to next month file
# Create a single dataframe for the year combining all the monthly data

# Select the folder having data files
import os

# Select the folder having data files
#os.chdir('/content/Assignments/EDA/data_NYC_Taxi/trip_records')

# Create a list of all the twelve files to read
#file_list = os.listdir()

# initialise an empty dataframe
#df = pd.DataFrame()


# iterate through the list of files and sample one by one:
for file_name in files:
  try:
    if file_name.endswith(".parquet"):
      df = pd.read_parquet(os.path.join(file_path, file_name))
      df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
      df['date'] = df['tpep_pickup_datetime'].dt.date
      df['hour'] = df['tpep_pickup_datetime'].dt.hour
      sampled_data = pd.DataFrame()

      for date in df['date'].unique():
        date_data = df[df['date'] == date]
        # Iterate through each hour of the selected date
        for hour in range(24):
          hour_data = date_data[date_data['hour'] == hour]
          # Sample 1% of the hourly data randomly
          if not hour_data.empty:
            sample = hour_data.sample(frac=0.0075, random_state=42)
            sampled_data = pd.concat([sampled_data, sample],ignore_index=True)
      sampled_df = pd.concat([sampled_df, sampled_data],ignore_index=True)
  except Exception as e:
        print(f"Error reading file {file_name}: {e}")

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._boots

After combining the data files into one DataFrame, convert the new DataFrame to a CSV or parquet file and store it to use directly.

Ideally, you can try keeping the total entries to around 250,000 to 300,000.

In [ ]:
# Store the df in csv/parquet
# df.to_parquet('')
sampled_df.to_parquet('sampled_data.parquet')

## **2** Data Cleaning
<font color = red>[30 marks]</font> <br>

Now we can load the new data directly.

In [ ]:
# Load the new data file
dfn = pd.read_parquet('sampled_data.parquet')



In [ ]:
# df.head()
dfn.head(5)

In [ ]:
# df.info()
dfn.info()

#### **2.1** Fixing Columns
<font color = red>[10 marks]</font> <br>

Fix/drop any columns as you seem necessary in the below sections

**2.1.1** <font color = red>[2 marks]</font> <br>

Fix the index and drop unnecessary columns

In [ ]:
# Fix the index and drop any columns that are not needed

dfn.reset_index(drop=True, inplace=True)
dfn.head()

**2.1.2** <font color = red>[3 marks]</font> <br>
There are two airport fee columns. This is possibly an error in naming columns. Let's see whether these can be combined into a single column.

In [ ]:
# Combine the two airport fee columns
if 'Airport_fee' in dfn.columns and 'airport_fee' in dfn.columns:
  dfn['airport_fee'] = dfn['Airport_fee'].combine_first(dfn['airport_fee'])
dfn.drop(columns = ['Airport_fee'] , inplace =True)
dfn.info()


**2.1.3** <font color = red>[5 marks]</font> <br>
Fix columns with negative (monetary) values

In [ ]:
# check where values of fare amount are negative

dfn[dfn['fare_amount']< 0]


Did you notice something different in the `RatecodeID` column for above records?

In [ ]:
# Analyse RatecodeID for the negative fare amounts
print(dfn['RatecodeID'].unique())
dfn[dfn['RatecodeID'] <0]


In [ ]:
# Find which columns have negative values

numeric_df = dfn.select_dtypes(include=['number'])
negative_columns = numeric_df.columns[(numeric_df < 0).any()]
negative_columns

In [ ]:
# fix these negative values
dfn[numeric_df.columns] = numeric_df[numeric_df.columns].clip(lower=0)

dfn[numeric_df.columns] = numeric_df[numeric_df.columns].applymap(lambda x: np.nan if x < 0 else x)


### **2.2** Handling Missing Values
<font color = red>[10 marks]</font> <br>

**2.2.1**  <font color = red>[2 marks]</font> <br>
Find the proportion of missing values in each column




In [ ]:
# Find the proportion of missing values in each column
missing_values= dfn.isnull().mean().sort_values(ascending=False)
missing_values


**2.2.2**  <font color = red>[3 marks]</font> <br>
Handling missing values in `passenger_count`

In [ ]:
# Display the rows with null values
# Impute NaN values in 'passenger_count'
null_rows = dfn[dfn['passenger_count'].isnull()]
null_rows

Did you find zeroes in passenger_count? Handle these.

**2.2.3**  <font color = red>[2 marks]</font> <br>
Handle missing values in `RatecodeID`

In [ ]:
# Fix missing values in 'RatecodeID'
dfn['RatecodeID'].isnull().sum()
mode_value = df['RatecodeID'].mode()[0]
dfn['RatecodeID'].fillna(mode_value, inplace=True)

**2.2.4**  <font color = red>[3 marks]</font> <br>
Impute NaN in `congestion_surcharge`

In [ ]:
# handle null values in congestion_surcharge
dfn[dfn['congestion_surcharge'].isnull()]
mode_value = dfn['congestion_surcharge'].mode()[0]
dfn['congestion_surcharge'].fillna(mode_value, inplace=True)
dfn['congestion_surcharge']



Are there missing values in other columns? Did you find NaN values in some other set of columns? Handle those missing values below.

In [ ]:
# Handle any remaining missing values

missing_summary = dfn.isnull().sum()
missing_value_column = missing_summary[missing_summary > 0]
print(missing_value_column)

### **2.3** Handling Outliers
<font color = red>[10 marks]</font> <br>

Before we start fixing outliers, let's perform outlier analysis.

In [ ]:
# Describe the data and check if there are any potential outliers present
# Check for potential out of place values in various columns

dfn['fare_amount']
df['payment_type'].unique()
df['trip_distance'].unique()
df['tip_amount']
Q1 = dfn['tip_amount'].quantile(0.25)
Q3 = dfn['tip_amount'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers_tip = dfn[(dfn['tip_amount'] < lower) | (dfn['tip_amount'] > upper)]
print(f"Tip amount outliers: {len(outliers_tip)} rows")


**2.3.1**  <font color = red>[10 marks]</font> <br>
Based on the above analysis, it seems that some of the outliers are present due to errors in registering the trips. Fix the outliers.

Some points you can look for:
- Entries where `trip_distance` is nearly 0 and `fare_amount` is more than 300
- Entries where `trip_distance` and `fare_amount` are 0 but the pickup and dropoff zones are different (both distance and fare should not be zero for different zones)
- Entries where `trip_distance` is more than 250  miles.
- Entries where `payment_type` is 0 (there is no payment_type 0 defined in the data dictionary)

These are just some suggestions. You can handle outliers in any way you wish, using the insights from above outlier analysis.

How will you fix each of these values? Which ones will you drop and which ones will you replace?

First, let us remove 7+ passenger counts as there are very less instances.

In [ ]:
# remove passenger_count > 6
filtered_df = dfn[dfn['passenger_count'] <= 6]


In [ ]:
# Continue with outlier handling
print(dfn.describe())


In [ ]:
# Do any columns need standardising?

dfn = dfn[~((dfn['trip_distance']<0.01)& (dfn['fare_amount']>300))]

dfn = dfn[~((dfn['trip_distance'] == 0) & (dfn['fare_amount'] ==0) & (dfn['PULocationID'] != dfn['DOLocationID'] ))]

dfn = dfn[dfn['trip_distance'] <= 250]
dfn= dfn[dfn['payment_type'] !=0]

In [ ]:
cl_to_scale = ['trip_distance','fare_amount', 'total_amount', 'tolls_amount', 'tip_amount']
for cl in cl_to_scale:
  dfn[cl] = (dfn[cl] - dfn[cl].min()) / (dfn[cl].max() - dfn[cl].min())

## **3** Exploratory Data Analysis
<font color = red>[90 marks]</font> <br>

In [10]:
dfn.columns.tolist()

NameError: name 'dfn' is not defined

#### **3.1** General EDA: Finding Patterns and Trends
<font color = red>[40 marks]</font> <br>

**3.1.1** <font color = red>[3 marks]</font> <br>
Categorise the varaibles into Numerical or Categorical.
* `VendorID`:
* `tpep_pickup_datetime`:
* `tpep_dropoff_datetime`:
* `passenger_count`:
* `trip_distance`:
* `RatecodeID`:
* `PULocationID`:
* `DOLocationID`:
* `payment_type`:
* `pickup_hour`:
* `trip_duration`:


The following monetary parameters belong in the same category, is it categorical or numerical?


* `fare_amount`
* `extra`
* `mta_tax`
* `tip_amount`
* `tolls_amount`
* `improvement_surcharge`
* `total_amount`
* `congestion_surcharge`
* `airport_fee`

In [ ]:
categorical_cols = dfn.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = dfn.select_dtypes(include=['number']).columns.tolist()


print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

##### Temporal Analysis

**3.1.2** <font color = red>[5 marks]</font> <br>
Analyse the distribution of taxi pickups by hours, days of the week, and months.

In [ ]:
# Find and show the hourly trends in taxi pickups

dfn['pickup_hour'] = dfn['tpep_pickup_datetime'].dt.hour

plt.figure(figsize=(12, 6))
sns.countplot(x='pickup_hour', data=dfn, palette='viridis')

plt.title("Taxi Pickups by Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Number of Pickups")
plt.xticks(range(24))
plt.show()

In [ ]:
# Find and show the daily trends in taxi pickups (days of the week)

dfn['date'] = pd.to_datetime(dfn['date'])
dfn['day_of_week'] = dfn['date'].dt.day_name()
plt.figure(figsize=(10, 6))
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.countplot(x='day_of_week', data=dfn, order=order, palette='coolwarm')
plt.title("Taxi Pickups by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Number of Pickups")
plt.show()

In [ ]:
# Show the monthly trends in pickups

dfn['pickup_month'] = dfn['date'].dt.month_name()
plt.figure(figsize=(10, 6))
months_order = ['January', 'February', 'March', 'April', 'May', 'June',
                'July', 'August', 'September', 'October', 'November', 'December']
sns.countplot(x='pickup_month', data=dfn, order=months_order, palette='cubehelix')
plt.title("Taxi Pickups by Month")
plt.xlabel("Month")
plt.ylabel("Number of Pickups")
plt.xticks(rotation=45)
plt.show()

##### Financial Analysis

Take a look at the financial parameters like `fare_amount`, `tip_amount`, `total_amount`, and also `trip_distance`. Do these contain zero/negative values?

In [2]:
# Analyse the above parameters
columns = ['fare_amount', 'tip_amount', 'total_amount', 'trip_distance']
for col in columns:
    zero_count = (dfn[col] == 0).sum()
    neg_count = (dfn[col] < 0).sum()
    print(f"{col} — Zero values: {zero_count}, Negative values: {neg_count}")



Do you think it is beneficial to create a copy DataFrame leaving out the zero values from these?

**3.1.3** <font color = red>[2 marks]</font> <br>
Filter out the zero values from the above columns.

**Note:** The distance might be 0 in cases where pickup and drop is in the same zone. Do you think it is suitable to drop such cases of zero distance?

In [3]:
# Create a df with non zero entries for the selected parameters.
new_df = dfn[dfn['fare_amount']>0 & (dfn['tip_amount'] >= 0) &
    (dfn['total_amount'] > 0) &
    (dfn['trip_distance'] > 0)].copy()


**3.1.4** <font color = red>[3 marks]</font> <br>
Analyse the monthly revenue (`total_amount`) trend

In [4]:
# Group data by month and analyse monthly revenue
new_df['total_amount'] = pd.to_numeric(new_df['total_amount'], errors='coerce')
new_df['pickup_month'] = new_df['date'].dt.month_name()
monthly_revenue = new_df.groupby('pickup_month')['total_amount'].sum().reset_index()
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
            'July', 'August', 'September', 'October', 'November', 'December']
monthly_revenue['pickup_month'] = pd.Categorical(monthly_revenue['pickup_month'], categories=month_order, ordered=True)
monthly_revenue = monthly_revenue.sort_values('pickup_month')


plt.figure(figsize=(10, 6))
plt.plot(monthly_revenue['pickup_month'], monthly_revenue['total_amount'], marker='o')
plt.title('Monthly Revenue Trend (Total Amount)')
plt.xlabel('Month')
plt.ylabel('Total Revenue ($)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



**3.1.5** <font color = red>[3 marks]</font> <br>
Show the proportion of each quarter of the year in the revenue

In [5]:
# Calculate proportion of each quarter

month_to_quarter = {
    'January': 'Q1', 'February': 'Q1', 'March': 'Q1',
    'April': 'Q2', 'May': 'Q2', 'June': 'Q2',
    'July': 'Q3', 'August': 'Q3', 'September': 'Q3',
    'October': 'Q4', 'November': 'Q4', 'December': 'Q4'
}

In [ ]:
new_df['quarter'] = new_df['pickup_month'].map(month_to_quarter)
quarterly_revenue = new_df.groupby('quarter')['total_amount'].sum().reset_index()
print(quarterly_revenue)
total_revenue = quarterly_revenue['total_amount'].sum()
quarterly_revenue['revenue_proportion'] = quarterly_revenue['total_amount'] / total_revenue
print(quarterly_revenue)

**3.1.6** <font color = red>[3 marks]</font> <br>
Visualise the relationship between `trip_distance` and `fare_amount`. Also find the correlation value for these two.

**Hint:** You can leave out the trips with trip_distance = 0

In [6]:
# Show how trip fare is affected by distance

df_filtered = new_df[(new_df['trip_distance'] > 0) & (new_df['fare_amount'] > 0)]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_filtered, x='trip_distance', y='fare_amount', alpha=0.3)
plt.title('Relationship Between Trip Distance and Fare Amount')
plt.xlabel('Trip Distance (miles)')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
correlation = df_filtered['trip_distance'].corr(df_filtered['fare_amount'])
correlation

**3.1.7** <font color = red>[5 marks]</font> <br>
Find and visualise the correlation between:
1. `fare_amount` and trip duration (pickup time to dropoff time)
2. `fare_amount` and `passenger_count`
3. `tip_amount` and `trip_distance`

In [7]:
# Show relationship between fare and trip duration

new_df['tpep_pickup_datetime'] = pd.to_datetime(new_df['tpep_pickup_datetime'])
new_df['tpep_dropoff_datetime'] = pd.to_datetime(new_df['tpep_dropoff_datetime'])
new_df['trip_duration_min'] = (new_df['tpep_dropoff_datetime'] - new_df['tpep_pickup_datetime']).dt.total_seconds() / 60
new_df_duration_filtered = new_df[(new_df['trip_duration_min'] > 0) & (new_df['fare_amount'] > 0)]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=new_df_duration_filtered, x='trip_duration_min', y='fare_amount', alpha=0.3)
plt.title('Fare Amount vs Trip Duration')
plt.xlabel('Trip Duration (minutes)')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [8]:
# Show relationship between fare and number of passengers
new_df_passenger_filtered = new_df[(new_df['passenger_count'] > 0) & (new_df['fare_amount'] > 0)]
plt.figure(figsize=(8, 6))
sns.boxplot(data=new_df_passenger_filtered, x='passenger_count', y='fare_amount')
plt.title('Fare Amount by Passenger Count')
plt.xlabel('Number of Passengers')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()


In [9]:
# Show relationship between tip and trip distance

new_df_tip_filtered = new_df[(new_df['trip_distance'] > 0) & (new_df['tip_amount'] > 0)]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=new_df_tip_filtered, x='trip_distance', y='tip_amount', alpha=0.3)
plt.title('Tip Amount vs Trip Distance')
plt.xlabel('Trip Distance (miles)')
plt.ylabel('Tip Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.show()

**3.1.8** <font color = red>[3 marks]</font> <br>
Analyse the distribution of different payment types (`payment_type`)

In [10]:
# Analyse the distribution of different payment types (payment_type).
payment_mapping = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute'
}
new_df['payment_type_desc'] = new_df['payment_type'].map(payment_mapping)
payment_counts = new_df['payment_type_desc'].value_counts().reset_index()
payment_counts.columns = ['Payment Type', 'Count']
plt.figure(figsize=(8, 6))
sns.barplot(data=payment_counts, x='Payment Type', y='Count', palette='pastel')
plt.title('Distribution of Payment Types')
plt.ylabel('Number of Trips')
plt.xlabel('Payment Type')
plt.xticks(rotation=30)
plt.grid(axis='y')
plt.tight_layout()
plt.show()



- 1= Credit card
- 2= Cash
- 3= No charge
- 4= Dispute



##### Geographical Analysis

For this, you have to use the *taxi_zones.shp* file from the *taxi_zones* folder.

There would be multiple files inside the folder (such as *.shx, .sbx, .sbn* etc). You do not need to import/read any of the files other than the shapefile, *taxi_zones.shp*.

Do not change any folder structure - all the files need to be present inside the folder for it to work.

The folder structure should look like this:
```
Taxi Zones
|- taxi_zones.shp.xml
|- taxi_zones.prj
|- taxi_zones.sbn
|- taxi_zones.shp
|- taxi_zones.dbf
|- taxi_zones.shx
|- taxi_zones.sbx

 ```

 You only need to read the `taxi_zones.shp` file. The *shp* file will utilise the other files by itself.

We will use the *GeoPandas* library for geopgraphical analysis
```
import geopandas as gpd
```

More about geopandas and shapefiles: [About](https://geopandas.org/en/stable/about.html)


Reading the shapefile is very similar to *Pandas*. Use `gpd.read_file()` function to load the data (*taxi_zones.shp*) as a GeoDataFrame. Documentation: [Reading and Writing Files](https://geopandas.org/en/stable/docs/user_guide/io.html)

In [ ]:
# !pip install geopandas

**3.1.9** <font color = red>[2 marks]</font> <br>
Load the shapefile and display it.

In [ ]:
# import geopandas as gpd

import geopandas as gpd
# Read the shapefile using geopandas
for shp_file in files:
  if shp_file.endswith(".shp"):
    print(shp_file)
    shp_file_path = os.path.join(file_path, shp_file)
    zones = gpd.read_file(shp_file_path)
zones.head()

Now, if you look at the DataFrame created, you will see columns like: `OBJECTID`,`Shape_Leng`, `Shape_Area`, `zone`, `LocationID`, `borough`, `geometry`.
<br><br>

Now, the `locationID` here is also what we are using to mark pickup and drop zones in the trip records.

The geometric parameters like shape length, shape area and geometry are used to plot the zones on a map.

This can be easily done using the `plot()` method.

In [ ]:
# print(zones.info())
# zones.plot()
print(zones.info())
zones.plot(figsize=(10, 10), edgecolor='black')
plt.title("NYC Taxi Zones")
plt.show()

Now, you have to merge the trip records and zones data using the location IDs.



**3.1.10** <font color = red>[3 marks]</font> <br>
Merge the zones data into trip data using the `locationID` and `PULocationID` columns.

In [ ]:
# Merge zones and trip records using locationID and PULocationID
trips_with_pickup_zone = new_df.merge(zones, how='left', left_on='PULocationID', right_on='LocationID', suffixes=('', '_pickup'))

trips_with_zones = trips_with_pickup_zone.merge(zones, how='left', left_on='DOLocationID', right_on='LocationID', suffixes=('', '_dropoff'))


**3.1.11** <font color = red>[3 marks]</font> <br>
Group data by location IDs to find the total number of trips per location ID

In [ ]:
# Group data by location and calculate the number of trips
pickup_counts = new_df.groupby("PULocationID").size().reset_index(name="pickup_trip_count")
dropoff_counts = new_df.groupby("DOLocationID").size().reset_index(name="dropoff_trip_count")

print(trips_with_zones.head())
print(dropoff_counts.head())


**3.1.12** <font color = red>[2 marks]</font> <br>
Now, use the grouped data to add number of trips to the GeoDataFrame.

We will use this to plot a map of zones showing total trips per zone.

In [ ]:
# Merge trip counts back to the zones GeoDataFrame
pickup_counts.rename(columns={'PULocationID': 'LocationID'}, inplace=True)
zones_with_counts = zones.merge(pickup_counts, on='LocationID', how='left')
zones_with_counts['pickup_trip_count'] = zones_with_counts['pickup_trip_count'].fillna(0)




In [ ]:
zones_with_counts.plot(
    column='pickup_trip_count',
    cmap='OrRd',
    legend=True,
    figsize=(12, 12),
    edgecolor='black'
)
plt.title("Total Pickup Trips per NYC Taxi Zone")
plt.axis('off')
plt.show()

The next step is creating a color map (choropleth map) showing zones by the number of trips taken.

Again, you can use the `zones.plot()` method for this. [Plot Method GPD](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html#geopandas.GeoDataFrame.plot)

But first, you need to define the figure and axis for the plot.

`fig, ax = plt.subplots(1, 1, figsize = (12, 10))`

This function creates a figure (fig) and a single subplot (ax)

---

After setting up the figure and axis, we can proceed to plot the GeoDataFrame on this axis. This is done in the next step where we use the plot method of the GeoDataFrame.

You can define the following parameters in the `zones.plot()` method:
```
column = '',
ax = ax,
legend = True,
legend_kwds = {'label': "label", 'orientation': "<horizontal/vertical>"}
```

To display the plot, use `plt.show()`.

**3.1.13** <font color = red>[3 marks]</font> <br>
Plot a color-coded map showing zone-wise trips

In [ ]:
# Define figure and axis


# Plot the map and display it



In [ ]:
# can you try displaying the zones DF sorted by the number of trips?



Here we have completed the temporal, financial and geographical analysis on the trip records.

**Compile your findings from general analysis below:**

You can consider the following points:

* Busiest hours, days and months
* Trends in revenue collected
* Trends in quarterly revenue
* How fare depends on trip distance, trip duration and passenger counts
* How tip amount depends on trip distance
* Busiest zones


#### **3.2** Detailed EDA: Insights and Strategies
<font color = red>[50 marks]</font> <br>

Having performed basic analyses for finding trends and patterns, we will now move on to some detailed analysis focussed on operational efficiency, pricing strategies, and customer experience.

##### Operational Efficiency

Analyze variations by time of day and location to identify bottlenecks or inefficiencies in routes

**3.2.1** <font color = red>[3 marks]</font> <br>
Identify slow routes by calculating the average time taken by cabs to get from one zone to another at different hours of the day.

Speed on a route *X* for hour *Y* = (*distance of the route X / average trip duration for hour Y*)

In [ ]:
# Find routes which have the slowest speeds at different times of the day

dfn['trip_duration_min'] = (dfn['tpep_dropoff_datetime'] - dfn['tpep_pickup_datetime']).dt.total_seconds() / 60
df_filtered = dfn[(dfn['trip_duration_min'] > 1) & (dfn['trip_distance'] > 0)].copy()
df_filtered['trip_duration_hr'] = df_filtered['trip_duration_min'] / 60
df_filtered['route'] = df_filtered['PULocationID'].astype(str) + '-' + df_filtered['DOLocationID'].astype(str)
grouped = df_filtered.groupby(['route', 'pickup_hour']).agg({
    'trip_distance': 'mean',
    'trip_duration_hr': 'mean'
}).reset_index()
grouped['avg_speed_mph'] = grouped['trip_distance'] / grouped['trip_duration_hr']
slowest_routes_by_hour = grouped.loc[grouped.groupby('pickup_hour')['avg_speed_mph'].idxmin()].reset_index(drop=True)
slowest_routes_by_hour.sort_values('pickup_hour', inplace=True)
print(slowest_routes_by_hour[['pickup_hour', 'route', 'avg_speed_mph']])

How does identifying high-traffic, high-demand routes help us?

**3.2.2** <font color = red>[3 marks]</font> <br>
Calculate the number of trips at each hour of the day and visualise them. Find the busiest hour and show the number of trips for that hour.

In [ ]:
# Visualise the number of trips per hour and find the busiest hour
trips_per_hour = dfn['pickup_hour'].value_counts().sort_index()
trips_per_hour
busiest_hour = trips_per_hour.idxmax()
busiest_count = trips_per_hour.max()
plt.figure(figsize=(12, 6))
plt.plot(trips_per_hour.index, trips_per_hour.values, marker='o')
plt.title('Number of Trips per Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Trips')
plt.grid(True)
plt.xticks(range(0, 24))
plt.axvline(busiest_hour, color='red', linestyle='--', label=f'Busiest Hour: {busiest_hour}')
plt.legend()
plt.show()


Remember, we took a fraction of trips. To find the actual number, you have to scale the number up by the sampling ratio.

**3.2.3** <font color = red>[2 mark]</font> <br>
Find the actual number of trips in the five busiest hours

In [ ]:
# Scale up the number of trips

# Fill in the value of your sampling fraction and use that to scale up the numbers

top_5_hours = trips_per_hour.sort_values(ascending=False).head(5)
print("Top 5 busiest hours (from sample):")
print(top_5_hours)
sample_fraction = 0.05
scaled_top_5 = (top_5_hours / sample_fraction).astype(int)
print("\nEstimated actual number of trips in the 5 busiest hours:")
print(scaled_top_5)



**3.2.4** <font color = red>[3 marks]</font> <br>
Compare hourly traffic pattern on weekdays. Also compare for weekend.

In [ ]:
# Compare traffic trends for the week days and weekends
dfn['date'] = pd.to_datetime(dfn['date'])
dfn['weekday_name'] = dfn['date'].dt.day_name()
dfn['day_type'] = dfn['weekday_name'].apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday')
traffic_by_hour = dfn.groupby(['pickup_hour', 'day_type']).size().reset_index(name='trip_count')
pivot_df = traffic_by_hour.pivot(index='pickup_hour', columns='day_type', values='trip_count').fillna(0)
pivot_df.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Traffic Patterns by Hour and Day Type')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Trips')



What can you infer from the above patterns? How will finding busy and quiet hours for each day help us?

**3.2.5** <font color = red>[3 marks]</font> <br>
Identify top 10 zones with high hourly pickups. Do the same for hourly dropoffs. Show pickup and dropoff trends in these zones.

In [ ]:
# Find top 10 pickup and dropoff zones
top_pickup_zones = dfn['PULocationID'].value_counts().head(10).index.tolist()
top_dropoff_zones = dfn['DOLocationID'].value_counts().head(10).index.tolist()
top_dropoff_zones
pickup_hourly = dfn[dfn['PULocationID'].isin(top_pickup_zones)] \
    .groupby(['PULocationID', 'pickup_hour']).size().reset_index(name='pickup_count')
dropoff_hourly = dfn[dfn['DOLocationID'].isin(top_dropoff_zones)] \
    .groupby(['DOLocationID', 'pickup_hour']).size().reset_index(name='dropoff_count')




In [ ]:
plt.figure(figsize=(14, 6))
for zone in top_pickup_zones:
    zone_data = pickup_hourly[pickup_hourly['PULocationID'] == zone]
    plt.plot(zone_data['pickup_hour'], zone_data['pickup_count'], label=f'Zone {zone}')

plt.title('Hourly Pickup Trends in Top 10 Zones')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Pickups')
plt.xticks(range(0, 24))
plt.legend()
plt.grid(True)
plt.show()

In [1]:
plt.figure(figsize=(14, 6))
for zone in top_dropoff_zones:
    zone_data = dropoff_hourly[dropoff_hourly['DOLocationID'] == zone]
    plt.plot(zone_data['pickup_hour'], zone_data['dropoff_count'], label=f'Zone {zone}')

plt.title('Hourly Dropoff Trends in Top 10 Zones')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Dropoffs')
plt.xticks(range(0, 24))
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined

**3.2.6** <font color = red>[3 marks]</font> <br>
Find the ratio of pickups and dropoffs in each zone. Display the 10 highest (pickup/drop) and 10 lowest (pickup/drop) ratios.

In [ ]:
# Find the top 10 and bottom 10 pickup/dropoff ratios
zone_ratios = pd.concat([pickup_counts, dropoff_counts], axis=1).fillna(0)
zone_ratios.info()
zone_ratios = zone_ratios[zone_ratios['dropoff_trip_count'] > 0]
zone_ratios['pickup_dropoff_ratio'] = zone_ratios['pickup_trip_count'] / zone_ratios['dropoff_trip_count']
top_10_ratios = zone_ratios.sort_values(by='pickup_dropoff_ratio', ascending=False).head(10)
print("Top 10 Pickup/Dropoff Ratios:")
print(top_10_ratios[['pickup_trip_count', 'dropoff_trip_count', 'pickup_dropoff_ratio']])
bottom_10_ratios = zone_ratios.sort_values(by='pickup_dropoff_ratio', ascending=True).head(10)
print("\nBottom 10 Pickup/Dropoff Ratios:")
print(bottom_10_ratios[['pickup_trip_count', 'dropoff_trip_count', 'pickup_dropoff_ratio']])


**3.2.7** <font color = red>[3 marks]</font> <br>
Identify zones with high pickup and dropoff traffic during night hours (11PM to 5AM)

In [ ]:
# During night hours (11pm to 5am) find the top 10 pickup and dropoff zones
# Note that the top zones should be of night hours and not the overall top zones
night_hours = list(range(0, 6)) + [23]
night_df = dfn[dfn['pickup_hour'].isin(night_hours)]
night_pickups = night_df['PULocationID'].value_counts().head(10)
night_dropoffs = night_df['DOLocationID'].value_counts().head(10)
print("Top 10 Pickup Zones (Night Hours 11PM–5AM):")
print(night_pickups)

print("\nTop 10 Dropoff Zones (Night Hours 11PM–5AM):")
print(night_dropoffs)

Now, let us find the revenue share for the night time hours and the day time hours. After this, we will move to deciding a pricing strategy.

**3.2.8** <font color = red>[2 marks]</font> <br>
Find the revenue share for nighttime and daytime hours.

In [ ]:
# Filter for night hours (11 PM to 5 AM)
night_hours = list(range(0, 6)) + [23]
night_df = dfn[dfn['pickup_hour'].isin(night_hours)]
day_hours = list(range(6, 23))
day_df = dfn[dfn['pickup_hour'].isin(day_hours)]
night_revenue = night_df['total_amount'].sum()

day_revenue = day_df['total_amount'].sum()
total_revenue = dfn['total_amount'].sum()
night_revenue_share = (night_revenue / total_revenue) * 100
day_revenue_share = (day_revenue / total_revenue) * 100
print(f" Nighttime Revenue Share: {night_revenue_share:.2f}%")
print(f" Daytime Revenue Share: {day_revenue_share:.2f}%")



##### Pricing Strategy

**3.2.9** <font color = red>[2 marks]</font> <br>
For the different passenger counts, find the average fare per mile per passenger.

For instance, suppose the average fare per mile for trips with 3 passengers is 3 USD/mile, then the fare per mile per passenger will be 1 USD/mile.

In [ ]:
# Analyse the fare per mile per passenger for different passenger counts
dfn['fare_per_mile'] = dfn['fare_amount'] / dfn['trip_distance']
dfn['fare_per_mile_per_passenger'] = dfn['fare_per_mile'] / dfn['passenger_count']
df_filtered = dfn[dfn['passenger_count'] > 0]

avg_fare_per_passenger = df_filtered.groupby('passenger_count')['fare_per_mile_per_passenger'].mean().reset_index()
avg_fare_per_passenger = avg_fare_per_passenger.sort_values(by='passenger_count')
print(avg_fare_per_passenger)



**3.2.10** <font color = red>[3 marks]</font> <br>
Find the average fare per mile by hours of the day and by days of the week

In [ ]:
# Compare the average fare per mile for different days and for different times of the day
avg_fare_hourly = dfn.groupby('pickup_hour')['fare_per_mile'].mean().reset_index()
avg_fare_hourly = avg_fare_hourly.sort_values(by='pickup_hour')
print("Average Fare Per Mile by Hour of the Day:")
print(avg_fare_hourly)


In [ ]:
avg_fare_weekday = dfn.groupby('day_of_week')['fare_per_mile'].mean().reset_index()
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
avg_fare_weekday['day_of_week'] = pd.Categorical(avg_fare_weekday['day_of_week'], categories=ordered_days, ordered=True)
avg_fare_weekday = avg_fare_weekday.sort_values(by='day_of_week')

print("Average Fare Per Mile by Day of the Week:")
print(avg_fare_weekday)

In [ ]:
plt.figure(figsize=(14, 6))

# Hourly average fare plot
plt.subplot(1, 2, 1)
plt.plot(avg_fare_hourly['pickup_hour'], avg_fare_hourly['fare_per_mile'], marker='o', color='b')
plt.title('Average Fare Per Mile by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Fare Per Mile')
plt.xticks(range(24))

# Daily average fare plot
plt.subplot(1, 2, 2)
plt.plot(avg_fare_weekday['day_of_week'], avg_fare_weekday['fare_per_mile'], marker='o', color='r')
plt.title('Average Fare Per Mile by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Average Fare Per Mile')

plt.tight_layout()
plt.show()

**3.2.11** <font color = red>[3 marks]</font> <br>
Analyse the average fare per mile for the different vendors for different hours of the day

In [ ]:
# Compare fare per mile for different vendors
avg_fare_vendor_hour = dfn.groupby(['VendorID', 'pickup_hour'])['fare_per_mile'].mean().reset_index()


print("Average Fare Per Mile by Vendor and Hour of the Day:")
print(avg_fare_vendor_hour)
plt.figure(figsize=(14, 6))

for vendor in avg_fare_vendor_hour['VendorID'].unique():
    vendor_data = avg_fare_vendor_hour[avg_fare_vendor_hour['VendorID'] == vendor]
    plt.plot(vendor_data['pickup_hour'], vendor_data['fare_per_mile'], marker='o', label=f"Vendor {vendor}")

plt.title('Average Fare Per Mile by Vendor and Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Fare Per Mile')
plt.xticks(range(24))
plt.legend(title="Vendor ID")
plt.tight_layout()
plt.show()


**3.2.12** <font color = red>[5 marks]</font> <br>
Compare the fare rates of the different vendors in a tiered fashion. Analyse the average fare per mile for distances upto 2 miles. Analyse the fare per mile for distances from 2 to 5 miles. And then for distances more than 5 miles.


In [ ]:
# Defining distance tiers
def assign_distance_tier(distance):
    if distance <= 2:
        return 'Tier 1: <= 2 miles'
    elif 2 < distance <= 5:
        return 'Tier 2: 2-5 miles'
    else:
        return 'Tier 3: > 5 miles'
dfn['distance_tier'] = dfn['trip_distance'].apply(assign_distance_tier)
df['fare_per_mile'] = df['fare_amount'] / df['trip_distance']
avg_fare_vendor_tier = dfn.groupby(['VendorID', 'distance_tier'])['fare_per_mile'].mean().reset_index()

print("Average Fare Per Mile by Vendor and Distance Tier:")
print(avg_fare_vendor_tier)
plt.figure(figsize=(14, 8))
sns.barplot(x='distance_tier', y='fare_per_mile', hue='VendorID', data=avg_fare_vendor_tier)
plt.title('Average Fare Per Mile by Vendor and Distance Tier')
plt.xlabel('Distance Tier')
plt.ylabel('Average Fare Per Mile')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



##### Customer Experience and Other Factors

**3.2.13** <font color = red>[5 marks]</font> <br>
Analyse average tip percentages based on trip distances, passenger counts and time of pickup. What factors lead to low tip percentages?

In [ ]:
#  Analyze tip percentages based on distances, passenger counts and pickup times

dfn['tip_percentage'] = (dfn['tip_amount'] / dfn['total_amount']) * 100
def categorize_trip_distance(distance):
    if distance <= 2:
        return 'Short (<=2 miles)'
    elif 2 < distance <= 5:
        return 'Medium (2-5 miles)'
    else:
        return 'Long (>5 miles)'
dfn['distance_category'] = dfn['trip_distance'].apply(categorize_trip_distance)
avg_tip_distance = dfn.groupby('distance_category')['tip_percentage'].mean().reset_index()

print("Average Tip Percentage by Distance Category:")
print(avg_tip_distance)
avg_tip_passenger_count = dfn.groupby('passenger_count')['tip_percentage'].mean().reset_index()
avg_tip_hour = dfn.groupby('pickup_hour')['tip_percentage'].mean().reset_index()



Additional analysis [optional]: Let's try comparing cases of low tips with cases of high tips to find out if we find a clear aspect that drives up the tipping behaviours

In [ ]:
# Compare trips with tip percentage < 10% to trips with tip percentage > 25%
low_tip_df = dfn[dfn['tip_percentage'] < 10]
high_tip_df = dfn[dfn['tip_percentage'] > 25]
print("Low Tip Trips (<10%)")
print(low_tip_df[['trip_distance', 'total_amount', 'passenger_count', 'pickup_hour']].describe())

print("\nHigh Tip Trips (>25%)")
print(high_tip_df[['trip_distance', 'total_amount', 'passenger_count', 'pickup_hour']].describe())


**3.2.14** <font color = red>[3 marks]</font> <br>
Analyse the variation of passenger count across hours and days of the week.

In [ ]:
# See how passenger count varies across hours and days
avg_passenger_by_hour = dfn.groupby('pickup_hour')['passenger_count'].mean().reset_index()
avg_passenger_by_day = dfn.groupby('day_of_week')['passenger_count'].mean().reset_index()


pivot_table = dfn.pivot_table(
    index='day_of_week',
    columns='pickup_hour',
    values='passenger_count',
    aggfunc='mean'
)



In [ ]:
sns.set(style="whitegrid")

# Line plot: average passenger count by hour
plt.figure(figsize=(12, 5))
sns.lineplot(x='pickup_hour', y='passenger_count', data=avg_passenger_by_hour, marker='o')
plt.title('Average Passenger Count by Hour of the Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Passenger Count')
plt.xticks(range(0, 24))
plt.grid(True)
plt.show()

# Bar plot: average passenger count by day
plt.figure(figsize=(10, 5))
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.barplot(x='day_of_week', y='passenger_count', data=avg_passenger_by_day, order=order)
plt.title('Average Passenger Count by Day of the Week')
plt.xlabel('Day of Week')
plt.ylabel('Average Passenger Count')
plt.xticks(rotation=45)
plt.show()

# Heatmap: average passenger count by day and hour
plt.figure(figsize=(14, 6))
sns.heatmap(pivot_table.loc[order], annot=True, fmt=".2f", cmap="YlGnBu")
plt.title('Heatmap of Average Passenger Count by Day and Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Day of Week')
plt.show()

**3.2.15** <font color = red>[2 marks]</font> <br>
Analyse the variation of passenger counts across zones

In [ ]:
# How does passenger count vary across zones
passenger_by_pu_zone = dfn.groupby('PULocationID')['passenger_count'].sum().reset_index().rename(columns={'passenger_count': 'total_passengers'})

passenger_by_do_zone = dfn.groupby('DOLocationID')['passenger_count'].sum().reset_index().rename(columns={'passenger_count': 'total_passengers'})




In [ ]:
# For a more detailed analysis, we can use the zones_with_trips GeoDataFrame
# Create a new column for the average passenger count in each zone.
avg_passenger_by_zone = dfn.groupby('PULocationID')['passenger_count'].mean().reset_index()
avg_passenger_by_zone.rename(columns={'PULocationID': 'LocationID', 'passenger_count': 'avg_passenger_count'}, inplace=True)

zones_with_trips = zones.merge(avg_passenger_by_zone, on='LocationID', how='left')
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
zones_with_trips.plot(
    column='avg_passenger_count',
    ax=ax,
    legend=True,
    cmap='OrRd',
    legend_kwds={'label': "Average Passenger Count", 'orientation': "vertical"}
)
plt.title("Average Passenger Count per Pickup Zone")
plt.axis('off')
plt.show()


Find out how often surcharges/extra charges are applied to understand their prevalance

**3.2.16** <font color = red>[5 marks]</font> <br>
Analyse the pickup/dropoff zones or times when extra charges are applied more frequently

In [ ]:
# How often is each surcharge applied?

dfn['extra_applied'] = dfn['extra'] > 0
dfn['mta_tax_applied'] = dfn['mta_tax'] > 0
dfn['improvement_applied'] = dfn['improvement_surcharge'] > 0
dfn['congestion_applied'] = dfn['congestion_surcharge'] > 0
dfn['airport_fee_applied'] = dfn['airport_fee'] > 0

surcharge_summary = {
    'Extra': dfn['extra_applied'].mean() * 100,
    'MTA Tax': dfn['mta_tax_applied'].mean() * 100,
    'Improvement Surcharge': dfn['improvement_applied'].mean() * 100,
    'Congestion Surcharge': dfn['congestion_applied'].mean() * 100,
    'Airport Fee': dfn['airport_fee_applied'].mean() * 100
}

surcharge_df = pd.DataFrame(surcharge_summary.items(), columns=['Surcharge Type', 'Percent of Trips'])
print(surcharge_df)

surcharges_by_hour = dfn.groupby('pickup_hour')[
    ['extra_applied', 'congestion_applied', 'airport_fee_applied']
].mean() * 100

surcharges_by_hour.plot(figsize=(12, 6), marker='o')
plt.title("Percentage of Trips with Surcharges by Hour")
plt.ylabel("Percent of Trips with Surcharge")
plt.xlabel("Hour of Day")
plt.grid(True)
plt.show()




In [ ]:
zone_surcharges = dfn.groupby('PULocationID')[['extra_applied', 'congestion_applied', 'airport_fee_applied']].mean().reset_index()
zone_surcharges.columns = ['LocationID', 'extra_freq', 'congestion_freq', 'airport_fee_freq']

## **4** Conclusion
<font color = red>[15 marks]</font> <br>

### **4.1** Final Insights and Recommendations
<font color = red>[15 marks]</font> <br>

Conclude your analyses here. Include all the outcomes you found based on the analysis.

Based on the insights, frame a concluding story explaining suitable parameters such as location, time of the day, day of the week etc. to be kept in mind while devising a strategy to meet customer demand and optimise supply.

**4.1.1** <font color = red>[5 marks]</font> <br>
Recommendations to optimize routing and dispatching based on demand patterns and operational inefficiencies

**4.1.2** <font color = red>[5 marks]</font> <br>

Suggestions on strategically positioning cabs across different zones to make best use of insights uncovered by analysing trip trends across time, days and months.

In [ ]:
Morning Rush (7–10 AM): Position cabs in residential areas like Brooklyn and Queens to capture commuter demand.

Evening Rush (4–8 PM): Shift to commercial zones such as Midtown and the Financial District.

Late Night (11 PM–3 AM): Focus on nightlife hotspots, airports, and major transit hubs.

Weekends: Reallocate to leisure and entertainment zones with late-night demand.

Monthly/Seasonal Trends: Increase coverage in tourist areas during holidays and recreational zones in summer.
Predictive Deployment: Use past data to pre-position cabs ahead of expected demand surges.

**4.1.3** <font color = red>[5 marks]</font> <br>
Propose data-driven adjustments to the pricing strategy to maximize revenue while maintaining competitive rates with other vendors.

In [ ]:
Use real-time data to adjust pricing dynamically.Monitor vendor-specific trends to stay within competitive benchmarks.
Test pricing changes in select zones/hours before scaling.